In [1]:
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('sample.csv')

In [4]:
df.head()

,record,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,ssn_count_30,...,ssn_homephone_count_0_by_3,ssn_homephone_count_0_by_7,ssn_homephone_count_0_by_14,ssn_homephone_count_0_by_30,ssn_homephone_count_1_by_3,ssn_homephone_count_1_by_7,ssn_homephone_count_1_by_14,ssn_homephone_count_1_by_30,ssn_name_count_0_by_3,ssn_name_count_0_by_7
0,1,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
1,2,1,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
2,3,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
3,4,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
4,5,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0


In [38]:
df.columns

Index(['dow_risk', 'ssn_day_since', 'ssn_count_0', 'ssn_count_1',
       'ssn_count_3', 'ssn_count_7', 'ssn_count_14', 'ssn_count_30',
       'address_day_since', 'address_count_0',
       ...
       'ssn_homephone_count_0_by_3', 'ssn_homephone_count_0_by_7',
       'ssn_homephone_count_0_by_14', 'ssn_homephone_count_0_by_30',
       'ssn_homephone_count_1_by_3', 'ssn_homephone_count_1_by_7',
       'ssn_homephone_count_1_by_14', 'ssn_homephone_count_1_by_30',
       'ssn_name_count_0_by_3', 'ssn_name_count_0_by_7'],
      dtype='object', length=309)

In [8]:
std_scale = StandardScaler().fit(df)
df_std = pd.DataFrame(std_scale.transform(df), columns=df.columns)

In [ ]:
print(df_std.mean(axis=0))
print(df_std.std(axis=0))

In [19]:
y = df_std['fraud_label']

In [10]:
ks_stat = {}
for col in df_std.columns:
    ks = ks_2samp(y, df_std[col])
    ks_stat[col] = ks

In [11]:
sorted_ks_pvalue = sorted(ks_stat.items(), key=lambda x:x[1].pvalue, reverse=True)
sorted_ks_stat = sorted(ks_stat.items(), key=lambda x:x[1].statistic)

In [13]:
with open('ks_stat.csv', 'w') as f:
    f.write('\n'.join([f'{i[0]},{i[1].statistic},{i[1].pvalue}' for i in sorted_ks_stat]))

In [14]:
from sklearn.metrics import normalized_mutual_info_score

In [23]:
mi = {}
for col in df_std.columns:
    mi[col] = normalized_mutual_info_score(y, df_std[col])

In [24]:
sorted_mi = sorted(mi.items(), key=lambda x:x[1], reverse=True)

In [26]:
with open('mi_stat.csv', 'w') as f:
    f.write('\n'.join([f'{i[0]},{i[1]}' for i in sorted_mi]))

In [29]:
num = len(sorted_mi) // 2
mi_set = {i[0] for i in sorted_mi[:num]}
ks_set = {i[0] for i in sorted_ks_stat[:num]}